### Recursos

Problemas interesantes para Aprendizaje por refuerzo
 * Gymnasium: https://gymnasium.farama.org/environments/box2d/
 * Solución del Lunar Lander con DQN: https://shiva-verma.medium.com/solving-lunar-lander-openaigym-reinforcement-learning-785675066197
 * Otra solución: https://wingedsheep.com/lunar-lander-dqn/ y https://github.com/wingedsheep/blog/blob/main/lunar-lander-dqn/lunar_lander_dqn_blog.ipynb
 * The Nature of Code: https://youtu.be/lu5ul7z4icQ
 * Librería para neuroevolución: https://pypi.org/project/nevopy/

## Instalación

%pip install gymnasium  
%pip install gymnasium[box2d] 

### Acciones adicionales

#### En macos

pip uninstall swig  
xcode-select -—install (si no se tienen ya)  
pip install swig  / sudo port install swig-python
pip install 'gymnasium[box2d]' # en zsh hay que poner las comillas  

Si se da el error [NSCheapMutableString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead.  
Hacer  
OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES

#### en Windows

Si da error, se debe a la falta de la versión correcta de Microsoft Visual C++ Build Tools, que es una dependencia de Box2D. Para solucionar este problema, puede seguir los siguientes pasos:  
 * Descargar Microsoft Visual C++ Build Tools desde https://visualstudio.microsoft.com/visual-cpp-build-tools/.
 * Dentro de la app, seleccione la opción "Herramientas de compilación de C++" para instalar.
 * Reinicie su sesión en Jupyter Notebook.
 * Ejecute nuevamente el comando !pip install gymnasium[box2d] en la línea de comandos de su notebook.

También puede deberse a no tener swig instalado o a tener una versión incorrecta, en ese caso habrá que instalarlo con

%pip uninstall swig  
%pip install swig

Si nada funciona, la solución última es instalar anaconda y cambiar el kernel.

In [3]:
# prueba lunar lander por humano
# apartado A de la práctica

import gymnasium as gym

env = gym.make("LunarLander-v3", render_mode="rgb_array")

import numpy as np
import pygame
import gymnasium.utils.play

lunar_lander_keys = {
    (pygame.K_UP,): 2,
    (pygame.K_LEFT,): 1,
    (pygame.K_RIGHT,): 3,
}
gymnasium.utils.play.play(env, zoom=3, keys_to_action=lunar_lander_keys, noop=0)

In [4]:
# agente deliberativo
# Apartado B de la práctica

# observaciones [x, y, vx, vy, ang, vang, pataiz, parader]
# acciones [nada, derecho, central, izquierdo]

# IMPORTANTE: tiene que ser creado por vosotros mismos, no vale la solución de OpenAI!

def policy(observation):
    x, y, vx, vy, angle, angular_velocity, left_leg, right_leg = observation

    # Ajustar el ángulo
    if angle > 0.05:
        print('⮕', end='')
        return 3  # Motor derecho
    elif angle < -0.05:
        print('⬅︎', end='')
        return 1  # Motor izquierdo

    # Controlar la velocidad vertical
    if vy < -0.2:
        print('⬆︎', end='')
        return 2  # Motor principal
    elif vy > 0.1:
        print('–', end='')
        return 0 # no hacer nada.

    # Controlar la velocidad horizontal
    if vx > 0.1:
      print('⬅︎', end='')
      return 1
    elif vx < -0.1:
      print('⮕', end='')
      return 3

    print('–', end='')
    return 0 

In [ ]:
# Bucle de simulación
env = gym.make("LunarLander-v3", render_mode="human")

observation, info = env.reset()
terminated, truncated = False, False
total_reward = 0

while not (terminated or truncated):
    action = policy(observation)
    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    env.render() # Muestra el entorno en tiempo real

print(f"Recompensa total: {total_reward}")
env.close()

### Importante:

Después de la evolución, para poder probar la red obtenido, dejar en la varoable global model la red óptima encontrada

model = ****

In [26]:
# neuroevolución con MLP y algoritmo genético
# Apartado C de la práctica

import numpy as np
from MLP import MLP
import random

# Configuración del algoritmo genético
POPULATION_SIZE = 500
GENERATIONS = 250
MUTATION_RATE = 0.1

# Crear población inicial
population = [MLP([8, 6, 4]) for _ in range(POPULATION_SIZE)]

def evaluate_fitness(model):
    env = gym.make("LunarLander-v3")
    total_reward = 0
    for _ in range(5):  # Evaluar el modelo en 5 episodios
        observation, _ = env.reset()
        done = False
        while not done:
            action = np.argmax(model.forward(observation))
            observation, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated
    env.close()
    return total_reward

def crossover(parent1, parent2):
    ch1 = parent1.to_chromosome()
    ch2 = parent2.to_chromosome()
    point = random.randint(0, len(ch1) - 1)
    child_ch = ch1[:point] + ch2[point:]
    child = MLP([8, 6, 4])
    child.from_chromosome(child_ch)
    return child

def mutate(model):
    ch = model.to_chromosome()
    for i in range(len(ch)):
        if random.random() < MUTATION_RATE:
            ch[i] += np.random.normal(0, 0.1)
    model.from_chromosome(ch)

for generation in range(GENERATIONS):
    # Evaluar fitness
    fitness = [evaluate_fitness(model) for model in population]
    ranked_population = [model for _, model in sorted(zip(fitness, population), reverse=True)]

    # Seleccionar los mejores individuos
    population = ranked_population[:POPULATION_SIZE // 2]

    # Reproducción
    while len(population) < POPULATION_SIZE:
        parent1, parent2 = random.sample(population[:POPULATION_SIZE // 4], 2)
        child = crossover(parent1, parent2)
        mutate(child)
        population.append(child)

    print(f"Episodio {5*(generation + 1)}: Recompensa promedio  = {max(fitness)/5}")

# Guardar el mejor modelo
best_model = population[0]


Episodio 5: Recompensa promedio  = -84.10731468193967
Episodio 10: Recompensa promedio  = -41.919480066557625
Episodio 15: Recompensa promedio  = -53.49174413102128
Episodio 20: Recompensa promedio  = -24.76425482217428
Episodio 25: Recompensa promedio  = -66.69375351764094
Episodio 30: Recompensa promedio  = -56.14360547307092
Episodio 35: Recompensa promedio  = -65.25420515792993
Episodio 40: Recompensa promedio  = -29.15008316812643
Episodio 45: Recompensa promedio  = -53.76038052238876
Episodio 50: Recompensa promedio  = -54.15575862521911
Episodio 55: Recompensa promedio  = -62.0844239482484
Episodio 60: Recompensa promedio  = -36.60984321626369
Episodio 65: Recompensa promedio  = -39.223906291131335
Episodio 70: Recompensa promedio  = -44.54880639078018
Episodio 75: Recompensa promedio  = -49.242186256533884
Episodio 80: Recompensa promedio  = -42.81293629127
Episodio 85: Recompensa promedio  = 2.2663999545574685
Episodio 90: Recompensa promedio  = -25.26688113772241
Episodio 95:

In [29]:
import pickle  # Importar el módulo para guardar el modelo


with open("best_model.pkl", "wb") as file:
    pickle.dump(best_model, file)



In [27]:
import gymnasium as gym
import numpy as np

# Función para ejecutar el mejor modelo en un episodio
def run(env, model):
    observation, info = env.reset()
    racum = 0
    while True:
        action = np.argmax(model.forward(observation))  # Seleccionar la mejor acción
        observation, reward, terminated, truncated, info = env.step(action)
        racum += reward

        if terminated or truncated:
            r = (racum + 200) / 500
            print(f"Recompensa acumulada: {racum}, Fitness calculado: {r}")
            return racum

# Configurar el entorno para grabar videos
env = gym.make("LunarLander-v3", render_mode="human")


# Probar el mejor modelo en 5 episodios
for episode in range(5):
    print(f"Episodio {episode + 1}:")
    total_reward = run(env, best_model)  # Usar la función `run` con el mejor modelo
    print(f"Recompensa total del episodio {episode + 1}: {total_reward}")

env.close()

Episodio 1:
Recompensa acumulada: 288.41109682610664, Fitness calculado: 0.9768221936522132
Recompensa total del episodio 1: 288.41109682610664
Episodio 2:
Recompensa acumulada: 288.5613030492823, Fitness calculado: 0.9771226060985646
Recompensa total del episodio 2: 288.5613030492823
Episodio 3:
Recompensa acumulada: 289.2691087581363, Fitness calculado: 0.9785382175162727
Recompensa total del episodio 3: 289.2691087581363
Episodio 4:
Recompensa acumulada: 241.81610034007412, Fitness calculado: 0.8836322006801482
Recompensa total del episodio 4: 241.81610034007412
Episodio 5:
Recompensa acumulada: 284.1131360988793, Fitness calculado: 0.9682262721977586
Recompensa total del episodio 5: 284.1131360988793


---------------

In [30]:
# walker

env = gym.make("BipedalWalker-v3", render_mode="human") # hardcore=True
...

Ellipsis